<div id="colab_button">
  <h1>Under the hood</h1>
  <a target="_blank" href="https://colab.research.google.com/github/mithril-security/blindbox/blob/main/docs/docs/getting-started/under-the-hood.ipynb"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>

### Nitro enclaves with BlindBox API: Under the hood
___________________________________________

In the quick tour, we saw how we can use the BlindBox Whisper API to query the Whisper model with data protected during computation by a Nitro enclave environment.

Let's take a look at what is going on under the hood when we use the Whisper API.

### Mithril's BlindBox Nitro Enclave server
___________________________________________

When you use our Whisper API with Nitro enclaves, by default, you connect to the BlindBox Nitro server hosted by Mithril Security. The advantage of this is that it allows you to test our APIs for free and without spending any time on deployment.

Nitro Enclaves are an AWS technology and can be deployed on Amazon EC2 instances only. Our BlindBox server for Nitro enclaves is deployed on an [Amazon EC2 R6i Instance](https://aws.amazon.com/ec2/instance-types/r6i/).

If you want to deploy your own BlindBox API server for Nitro enclaves, check out our guide on how to do this [here].

Let's take a look at how our Nitro server is configured:

![nitro server architecture](../../assets/Nitro-server-arch.png)

Queries are sent to our AWS instance on port 443 as HTTP requests. They are then redirected to our enclave via the VSOCK channel which then handles the requests in our isolated enclave environment before returning the results via the VSOCK channel.

### Attestation

When we call the `transcribe` method in the BlindBox API, the first thing we are going to do is create a `BlindBox Connection` object using the `blindbox.core.connect()` method.

In [ ]:
!pip install blindbox

#  connect to your server instance
BLINDAI_NITRO_SERVER = "44.228.153.183"

client = blindbox.core.connect(
    addr=BLINDAI_NITRO_SERVER
)

 All this means is that we are going to establish a connection with our server instance, verify this server instance and then keep a track of this connection for following queries.

This `connect()` method is where we will perform attestation, the process of verifying the enclave application code, OS and settings, before allowing any data to be sent to the enclave. For more details about attestation, check out our Nitro attestation section in our [how we protect your data guide](./confidential_computing.md).

If we were to attempt to connect with an enclave running a modified version of the BlindBox API or a misconfigured enclave, we would see the following error:

![attestation error](../../assets/attestation-error.png)

### Querying the Whisper model

when we query the Whisper model on our Mithril Nitro enclave server, under the hood we are sending a request to our verified nitro enclave using a secure TLS communication channel. We use the `.text` attribute to specify that we want to get our results back in a string format.

The computation performed to get the result is performed within the Nitro enclave and the audio file and results are never accessible to anyone outside of the enclave.

In [ ]:
!pip install requests

import requests

res = requests.post(
    "https://nitro.mithrilsecurity.io/whisper/predict",
    files={
        "audio": open("test2.wav", "rb"),
    },
).text

print(res)

## Conclusions
_________________

In this guide, we have:

+ Seen how the Mithril **Nitro server is set-up**. 
+ Discussed how **attestation protects** your data. 
+ Discovered how **queries are performed** under the hood. 